In [31]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [32]:
#jwddf = pd.read_csv('D:/data/sun/dbscan/jwd_all.csv')
#jwddf = pd.read_csv('D:/data/sun/dbscan/jwd_all.csv')
jwddf = pd.read_csv('D:/data/sun/dbscan/out1.csv')
jwddf_groupby = jwddf.groupby(by = 'xq_code')
xq_codes = list(jwddf_groupby.groups.keys())

xqwz = pd.DataFrame({'xq_code':'','east_longitude':'','west_longitude':'','north_latitude':'','south_latitude':''},pd.Index(range(1)))
xq_code_list = []
pre_night_longitude_max_list = []
pre_night_longitude_min_list = []
pre_night_latitude_max_list = []
pre_night_latitude_min_list = []

In [33]:
def showAllPointImage(jwd,xq_code):
    plt.scatter(jwd['pre_night_longitude'], jwd['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def showCalPointImage(test_df,xq_code):
    plt.scatter(test_df['pre_night_longitude'], test_df['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def showClusterImage(core_member,non_core_member,noise_member,n_clusters_):
    plt.plot(core_member['pre_night_longitude'], core_member['pre_night_latitude'], '+', markersize=6)
    plt.plot(non_core_member['pre_night_longitude'], non_core_member['pre_night_latitude'], '^', markersize=6)
    plt.plot(noise_member['pre_night_longitude'], noise_member['pre_night_latitude'], '.', markersize=6)
    plt.title('Estimated number of clusters: %d' % n_clusters_)
    plt.show()
    
def showMemberImage(xy,xq_code):
    plt.scatter(xy['pre_night_longitude'], xy['pre_night_latitude'], c="red", marker='.', label=xq_code) 
    plt.xlabel('longitude')  
    plt.ylabel('latitude')  
    plt.legend(loc=2)  
    plt.show() 
    
def calMedian(jwd):
    longitude_zws = jwd['pre_night_longitude'].median()
    latitude_zws = jwd['pre_night_latitude'].median()
    return longitude_zws,latitude_zws
    
def calMean(jwd):
    longitude_pjs = jwd['pre_night_longitude'].mean()
    latitude_pjs = jwd['pre_night_latitude'].mean()
    return longitude_pjs,latitude_pjs

def filterExcepValues(jwd,xq_code):
    longitude_zws,latitude_zws = calMedian(jwd)
    test_df = jwd[(jwd.pre_night_longitude > longitude_zws - 0.025) &
        (jwd.pre_night_longitude < longitude_zws + 0.025) &
        (jwd.pre_night_latitude > latitude_zws - 0.025) &
        (jwd.pre_night_latitude < latitude_zws + 0.025)]
    
    excep_num = jwd.shape[0] - test_df.shape[0]
    print('xq_code：',xq_code ,':全部数据点：',jwd.shape[0],'个：可用数据点：',test_df.shape[0],'个：异常数据点:',excep_num,'个')
    return test_df

def dbscan(test_df,eps1,min_samples1):
    db = DBSCAN(eps=eps1, min_samples=min_samples1).fit(test_df[['pre_night_longitude','pre_night_latitude']])
    labels = db.labels_
    #计算簇个数
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    return db,n_clusters_,labels

In [34]:
i = 0
for xq_code in xq_codes:
    i += 1
    print("当前计算第",i,"个小区")
    jwd = jwddf.loc[jwddf['xq_code']==xq_code]
    
    #展示全部用户位置分布
    #showAllPointImage(jwd,xq_code)
    
    #剔除异常值
    test_df = filterExcepValues(jwd,xq_code)
    
    #展示参与计算的用户的位置分布
    #showCalPointImage(test_df,xq_code)

    if test_df.shape[0] != 0:
        db,n_clusters_,labels = dbscan(test_df,0.001,round(test_df.shape[0] * 0.4))
        
        if n_clusters_ == 1:
            core_samples_mask = np.zeros_like(labels, dtype=bool)
            #核心点
            core_samples_mask[db.core_sample_indices_] = True
            #成员点
            class_member_mask = (labels == 0)
            #噪声点
            noise_member_mask = (labels == -1)
            
            core_member = test_df[class_member_mask & core_samples_mask]
            non_core_member = test_df[class_member_mask & ~core_samples_mask]
            class_member = test_df[class_member_mask]
            noise_member = test_df[noise_member_mask]
            
            
            #展示簇分布
            #showClusterImage(core_member,non_core_member,noise_member,n_clusters_)
            #展示成员点分布
            #showMemberImage(class_member,xq_code)
            
            print('xq_code：',xq_code,',计算点数量：',test_df.shape[0],',成员点数量：',class_member.shape[0],',核心点数量：',core_member.shape[0],',非核心点数量：',non_core_member.shape[0],',噪音点数量：',noise_member.shape[0])
            print('xq_code：',xq_code,class_member['pre_night_longitude'].max(),class_member['pre_night_longitude'].min(),class_member['pre_night_latitude'].max(),class_member['pre_night_latitude'].min()) 
            
            east = class_member['pre_night_longitude'].max()
            west = class_member['pre_night_longitude'].min()
            north = class_member['pre_night_latitude'].max()
            south = class_member['pre_night_latitude'].min()
            
            if (east - west) > 0.0005 and (north - south) > 0.0005:
                xq_code_list.append(xq_code)
                pre_night_longitude_max_list.append(east)
                pre_night_longitude_min_list.append(west)
                pre_night_latitude_max_list.append(north)
                pre_night_latitude_min_list.append(south)
            else:
                pass
                #print('xq_code：',xq_code ,'小区范围大小，重新计算')

        else:
            pass
            #print('xq_code：',xq_code ,'计算簇个数为',n_clusters_,'个，稍后计算！')
            
    else:
        pass
        #print('xq_code：',xq_code ,':可用数据点为',test_df.shape[0],'个，稍后计算！')

当前计算第 1 个小区
xq_code： BUSINESS_COMMUNITY-0000000054995ac90154aa61edf80042 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-0000000054995ac90154aa61edf80042 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-0000000054995ac90154aa61edf80042 114.30478059999999 114.30478059999999 30.4173316 30.4173316
当前计算第 2 个小区
xq_code： BUSINESS_COMMUNITY-141450066916 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450066916 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450066916 114.14358329999999 114.14358329999999 30.62750362 30.62750362
当前计算第 3 个小区
xq_code： BUSINESS_COMMUNITY-141450067347 :全部数据点： 30 个：可用数据点： 24 个：异常数据点: 6 个
当前计算第 4 个小区
xq_code： BUSINESS_COMMUNITY-141450067362 :全部数据点： 34 个：可用数据点： 20 个：异常数据点: 14 个
当前计算第 5 个小区
xq_code： BUSINESS_COMMUNITY-141450067369 :全部数据点： 26 个：可用数据点： 23 个：异常数据点: 3 个
xq_code： BUSINESS_COMMUNITY-141450067369 ,计算点数量： 23 ,成员点数量： 12 ,核心点数量： 12 ,非核心点数量： 0 ,噪音点数量： 11
xq_code： BUSIN

xq_code： BUSINESS_COMMUNITY-141450070633 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 51 个小区
xq_code： BUSINESS_COMMUNITY-141450070638 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 52 个小区
xq_code： BUSINESS_COMMUNITY-141450070690 :全部数据点： 5 个：可用数据点： 0 个：异常数据点: 5 个
当前计算第 53 个小区
xq_code： BUSINESS_COMMUNITY-141450070704 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450070704 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450070704 114.3728169 114.3728169 30.60595945 30.60595945
当前计算第 54 个小区
xq_code： BUSINESS_COMMUNITY-141450070715 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450070715 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450070715 114.43052069999999 114.43052069999999 30.46392889 30.46392889
当前计算第 55 个小区
xq_code： BUSINESS_COMMUNITY-141450070762 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450070762 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS

xq_code： BUSINESS_COMMUNITY-141450072373 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450072373 114.27807450000002 114.27807450000002 30.57668327 30.57668327
当前计算第 94 个小区
xq_code： BUSINESS_COMMUNITY-141450072543 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 95 个小区
xq_code： BUSINESS_COMMUNITY-141450072548 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 96 个小区
xq_code： BUSINESS_COMMUNITY-141450072553 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 97 个小区
xq_code： BUSINESS_COMMUNITY-141450072558 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 98 个小区
xq_code： BUSINESS_COMMUNITY-141450072563 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450072563 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450072563 114.33513310000001 114.33513310000001 30.60181035 30.60181035
当前计算第 99 个小区
xq_code： BUSINESS_COMMUNITY-141450072717 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 100 个小区
xq_code： BUSINESS_COMMUNITY-141450072985 :全部数据点： 1 个：可用数据点： 1 个：异常数据

xq_code： BUSINESS_COMMUNITY-141450857006 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450857006 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450857006 114.24400259999999 114.24400259999999 30.53939474 30.53939474
当前计算第 148 个小区
xq_code： BUSINESS_COMMUNITY-141450857097 :全部数据点： 3 个：可用数据点： 0 个：异常数据点: 3 个
当前计算第 149 个小区
xq_code： BUSINESS_COMMUNITY-141450857130 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 150 个小区
xq_code： BUSINESS_COMMUNITY-141450857152 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450857152 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450857152 114.35561809999999 114.35561809999999 30.57697503 30.57697503
当前计算第 151 个小区
xq_code： BUSINESS_COMMUNITY-141450857160 :全部数据点： 4 个：可用数据点： 0 个：异常数据点: 4 个
当前计算第 152 个小区
xq_code： BUSINESS_COMMUNITY-141450857344 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450857344 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 

xq_code： BUSINESS_COMMUNITY-141450858964 :全部数据点： 4 个：可用数据点： 0 个：异常数据点: 4 个
当前计算第 199 个小区
xq_code： BUSINESS_COMMUNITY-141450858984 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-141450858984 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450858984 114.25851509999998 114.25851509999998 30.62163694 30.62163694
当前计算第 200 个小区
xq_code： BUSINESS_COMMUNITY-141450859000 :全部数据点： 6 个：可用数据点： 1 个：异常数据点: 5 个
xq_code： BUSINESS_COMMUNITY-141450859000 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450859000 114.34748219999999 114.34748219999999 30.51588502 30.51588502
当前计算第 201 个小区
xq_code： BUSINESS_COMMUNITY-141450859045 :全部数据点： 3 个：可用数据点： 1 个：异常数据点: 2 个
xq_code： BUSINESS_COMMUNITY-141450859045 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-141450859045 114.2272426 114.2272426 30.61939524 30.61939524
当前计算第 202 个小区
xq_code： BUSINESS_COMMUNITY-141450859106 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 

...

xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165470694275234 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165470694275234 114.1314213 114.1314213 30.47392377 30.47392377
当前计算第 1824 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165550d109254b4 :全部数据点： 8 个：可用数据点： 0 个：异常数据点: 8 个
当前计算第 1825 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165550f6c4454cc :全部数据点： 7 个：可用数据点： 0 个：异常数据点: 7 个
当前计算第 1826 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb01655580ca2e550c :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 1827 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165565ef15e557c :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 1828 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165568892eb55cc :全部数据点： 17 个：可用数据点： 13 个：异常数据点: 4 个
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165568892eb55cc ,计算点数量： 13 ,成员点数量： 10 ,核心点数量： 10 ,非核心点数量： 0 ,噪音点数量： 3
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165568892eb55cc 114.29677759999998 114.29609409999999 30.59216579 3

xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165ae22c0ee0332 ,计算点数量： 24 ,成员点数量： 17 ,核心点数量： 16 ,非核心点数量： 1 ,噪音点数量： 7
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165ae22c0ee0332 114.35894760000001 114.35788190000001 30.53192513 30.53067159
当前计算第 1869 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165ae3e689a0362 :全部数据点： 117 个：可用数据点： 90 个：异常数据点: 27 个
当前计算第 1870 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165b1c1902804ea :全部数据点： 47 个：可用数据点： 37 个：异常数据点: 10 个
当前计算第 1871 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165b27a3214053c :全部数据点： 303 个：可用数据点： 235 个：异常数据点: 68 个
当前计算第 1872 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d10646805eb0165b3923182068e :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 1873 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0165c610eef30068 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0165c610eef30068 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0165c610eef30068 114.53416100000001 114.5341

xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a46b98d85e30 :全部数据点： 47 个：可用数据点： 45 个：异常数据点: 2 个
当前计算第 1924 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a8e047d36038 :全部数据点： 43 个：可用数据点： 35 个：异常数据点: 8 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a8e047d36038 ,计算点数量： 35 ,成员点数量： 19 ,核心点数量： 17 ,非核心点数量： 2 ,噪音点数量： 16
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a8e047d36038 114.3150383 114.3139717 30.57521984 30.57341706
当前计算第 1925 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166a96aece961a8 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 1926 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166aa8ad6686298 :全部数据点： 139 个：可用数据点： 129 个：异常数据点: 10 个
当前计算第 1927 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166ae4bb5616488 :全部数据点： 47 个：可用数据点： 34 个：异常数据点: 13 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166ae4bb5616488 ,计算点数量： 34 ,成员点数量： 15 ,核心点数量： 15 ,非核心点数量： 0 ,噪音点数量： 19
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0166ae4bb5616488 114.3788031 114.3783935 30.60630576 30.60602692
当前计

xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016734d9c1ba1c00 ,计算点数量： 14 ,成员点数量： 9 ,核心点数量： 9 ,非核心点数量： 0 ,噪音点数量： 5
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc016734d9c1ba1c00 114.2302878 114.2302878 30.55637449 30.55637449
当前计算第 1975 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0167444022b82350 :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0167444022b82350 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc0167444022b82350 114.2359297 114.2359297 30.64393065 30.64393065
当前计算第 1976 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01674490081c2368 :全部数据点： 55 个：可用数据点： 49 个：异常数据点: 6 个
当前计算第 1977 个小区
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01674dfa9ec62468 :全部数据点： 210 个：可用数据点： 155 个：异常数据点: 55 个
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01674dfa9ec62468 ,计算点数量： 155 ,成员点数量： 98 ,核心点数量： 98 ,非核心点数量： 0 ,噪音点数量： 57
xq_code： BUSINESS_COMMUNITY-8a9e2d1065c3e6dc01674dfa9ec62468 114.29154390000001 114.290958 30.45618809 

xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba54015971d4144274e3 :全部数据点： 2 个：可用数据点： 0 个：异常数据点: 2 个
当前计算第 2020 个小区
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba54015972ca5f8542ef :全部数据点： 1 个：可用数据点： 1 个：异常数据点: 0 个
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba54015972ca5f8542ef ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba54015972ca5f8542ef 114.3337689 114.3337689 30.52214386 30.52214386
当前计算第 2021 个小区
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba540159b9fb2fbb2ed4 :全部数据点： 4 个：可用数据点： 3 个：异常数据点: 1 个
当前计算第 2022 个小区
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba540159bac1b3641e04 :全部数据点： 7 个：可用数据点： 1 个：异常数据点: 6 个
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba540159bac1b3641e04 ,计算点数量： 1 ,成员点数量： 1 ,核心点数量： 1 ,非核心点数量： 0 ,噪音点数量： 0
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba540159bac1b3641e04 114.34355900000001 114.34355900000001 30.56049492 30.56049492
当前计算第 2023 个小区
xq_code： BUSINESS_COMMUNITY-8a9e74c95952ba540159c782e3ad31ea :全部数据点： 5 个：可用数据点： 1 个：异常数据点: 4 个
xq_code： 

In [35]:
xqwz = pd.DataFrame(xq_code_list)
xqwz['east_longitude'] = pd.DataFrame(pre_night_longitude_max_list)
xqwz['west_longitude'] = pd.DataFrame(pre_night_longitude_min_list)
xqwz['north_latitude'] = pd.DataFrame(pre_night_latitude_max_list)
xqwz['south_latitude'] = pd.DataFrame(pre_night_latitude_min_list)

In [36]:
xqwz.to_csv('D:/data/sun/dbscan/xqwz.csv')
print('end')

end
